In [114]:
import math
import torch
import numpy as np

import nbimporter
from HelperFunctions import SparseMatrix, buildLaplacian

In [148]:
def myGradHarmonic(v: np.array, f: np.array, b: np.array, bc: np.array, k: int) -> np.array:
    """
    Inputs:
    v: Array of vertices
    f: Array of faces
    b: Array of indices for boundary vertices
    bc: Array of boundary values
    k: Power of harmonic operator
    ______________________________
    Output:
    w: Array of final vertices
    
    Note: If using displacements instead of the direct values, bc should be the displacement
    of the boundary conditions and w will be the resulting displacement of all values in v so
    final_v = w + v
    """
    
    dtype = torch.float64
    device = "cpu"
    torch.set_default_device(device)
    torch.set_default_dtype(dtype)
        
    #construct the cotangent Laplacian matrix
    l: SparseMatrix = buildLaplacian(v,f)
    #since this is harmonic don't forget to raise it to the kth power
    l = l.pow_sparse(k)
    
    #array of all vertices
    v_all: np.array = np.arange(v.shape[0])
    #array of non-boundary vertices
    v_in: np.array = np.setdiff1d(v_all,b)
    
    #slice up
    lii = l.slice_sparse(v_in,v_in)
    lib = l.slice_sparse(v_in,b.flatten())
    
    #get these guys into tensor form
    L_ii: torch.sparse_coo_tensor = torch.sparse_coo_tensor([lii.r,lii.c], lii.v, (lii.rs,lii.rs))
    L_ii = L_ii.coalesce()
    
    L_ib: torch.sparse_coo_tensor = torch.sparse_coo_tensor([lib.r,lib.c], lib.v, (lii.rs,len(b)))
    L_ib = L_ib.coalesce()
    
    #intialize x and x'
    win: torch.tensor = torch.tensor(np.zeros(v[v_in, :].shape), requires_grad=True)
    wb: torch.tensor = torch.tensor(bc)
    

    #tolerance is for the step size so it stops if I starting stepping this tiny
    tolerance: float = 1e-3
    learning_rate: float = 1e-4
    for i in range(20000):

        #calculate the loss
        Lii_w: torch.tensor = L_ii @ win
        Lib_w: torch.tensor = L_ib @ wb
        
        loss: torch.float = (Lii_w + Lib_w).pow(2).sum()

        #compute the gradient of loss with respect to win
        loss.backward()
        
        #to stop it if it blows up 
        if win.norm() > 1e6:
            raise RuntimeError("GD diverged—stopping early")
        
        
        old: torch.tensor = win.detach().clone()
        
        #make your 'tiny step' downhill
        with torch.no_grad():
            win -= learning_rate * win.grad
            
            #zero the gradient
            win.grad = None
        
        #check the step size
        step: float = (win.detach() - old).norm().item()
        if step < tolerance: 
            print("stopping at iter:" , i)
            break
        
        
    #remerge with the boundary vertices and return
    w: np.array = np.zeros_like(v)
    w[v_in] = win.detach().numpy()
    w[b.flatten()] = bc
    
    return w

In [145]:
def myBetterGradHarmonic(v: np.array, f: np.array, b: np.array, bc: np.array, k: int) -> np.array:
    """
    Inputs:
    v: Array of vertices
    f: Array of faces
    b: Array of indices for boundary vertices
    bc: Array of boundary values
    k: Power of harmonic operator
    ______________________________
    Output:
    w: Array of final vertices
    
    Note: If using displacements instead of the direct values, bc should be the displacement
    of the boundary conditions and w will be the resulting displacement of all values in v so
    final_v = w + v
    """
    
    dtype = torch.float64
    device = "cpu"
    torch.set_default_device(device)
    torch.set_default_dtype(dtype)
        
    #construct the cotangent Laplacian matrix
    l: SparseMatrix = buildLaplacian(v,f)
    #since this is harmonic don't forget to raise it to the kth power
    l = l.pow_sparse(k)
    
    #array of all vertices
    v_all: np.array = np.arange(v.shape[0])
    #array of non-boundary vertices
    v_in: np.array = np.setdiff1d(v_all,b)
    
    #slice up
    lii = l.slice_sparse(v_in,v_in)
    lib = l.slice_sparse(v_in,b.flatten())
    
    #get these guys into tensor form
    L_ii: torch.sparse_coo_tensor = torch.sparse_coo_tensor([lii.r,lii.c], lii.v, (lii.rs,lii.rs))
    L_ii = L_ii.coalesce()
    
    L_ib: torch.sparse_coo_tensor = torch.sparse_coo_tensor([lib.r,lib.c], lib.v, (lii.rs,len(b)))
    L_ib = L_ib.coalesce()
    
    #intialize x and x'
    win: torch.tensor = torch.tensor(np.zeros(v[v_in, :].shape), requires_grad=True)
    wb: torch.tensor = torch.tensor(bc)
    

    #tolerance is for the step size so it stops if I starting stepping this tiny
    tolerance: float = 1e-3
    learning_rate: float = 1e-3
    
    for i in range(20000):

        #calculate the loss
        Lii_w = L_ii @ win
        Lib_w = L_ib @ wb

        interior = 0.5 * torch.sum(win * Lii_w)
        boundary = torch.sum(win * Lib_w)
        loss     = interior + boundary
        
        loss: torch.float = interior + boundary
            
        #compute the gradient of loss with respect to win
        loss.backward()
        
        #to stop it if it blows up 
        if win.norm() > 1e6:
            raise RuntimeError("GD diverged—stopping early")
        
        
        old: torch.tensor = win.detach().clone()
        
        #make your 'tiny step' downhill
        with torch.no_grad():
            win -= learning_rate * win.grad
            
            #zero the gradient
            win.grad = None
        
        #check the step size
        step: float = (win.detach() - old).norm().item()
        if step < tolerance: 
            print("stopping at iter:" , i)
            break

    #remerge with the boundary vertices and return
    w: np.array = np.zeros_like(v)
    w[v_in] = win.detach().numpy()
    w[b.flatten()] = bc
    
    return w